In [30]:
import numpy as np
from scipy.special import logsumexp
import itertools
np.seterr(divide='ignore')


def assert_dist_non_negative(log_p):
    assert np.all(log_p <= 0)


def assert_dist_sums_to_1(log_p, axis, check_idx=None):
    if check_idx is None: assert np.all(np.isclose(np.exp(logsumexp(log_p, axis=axis)), 1))
    else: assert np.all(np.isclose(np.exp(logsumexp(log_p, axis=axis)[check_idx]), 1))


def _log_forward(data_obs, log_p, log_t_mat, log_e_mat, val_X, T):
    """
    F[1, k] = p(X_1 = k, o_1) = p(o_1 | X_1 = k) * p(X_1 = k) = e[k -> o_1] * p[k]
    F[t, k] = p(X_t = k, o_{1:t}) = e[k -> o_t] * sum_l[ F[t-1, l] * tau[l -> k] ]
    shape = N x T x |Val(X)|
    """
    N = data_obs.shape[0]
    log_F = np.zeros((N, T, len(val_X))) - np.inf
    log_F[:, 0] = log_p + log_e_mat[:, data_obs[:, 0]].T
    for t in range(1, T):
        temp = log_F[:, t - 1, None] + log_t_mat.T[None, :]
        log_F[:, t] = logsumexp(temp + log_e_mat[:, data_obs[:, t]].T[:, :, None], axis=2)

    # tests
    assert_dist_non_negative(log_F)
    return log_F


class HMM:
    def __init__(self, T, val_X, val_O, prior, transition_mat, emission_mat):
        self.T = T
        self.val_X = val_X
        self.val_O = val_O
        self.log_prior = np.log(prior)
        self.log_transition_mat = np.log(transition_mat)
        self.log_emission_mat = np.log(emission_mat)

    def get_CPDs(self):
        return {'prior': np.exp(self.log_prior),
                'transition_mat': np.exp(self.log_transition_mat),
                'emission_mat': np.exp(self.log_emission_mat)}

    def print_CPDs(self):
        cpds = self.get_CPDs()
        k = 'prior'
        print(k)
        print(np.array([f'prior({x})={cpds[k][x]:.3f}' for x in self.val_X]))
        k = 'transition_mat'
        print(k)
        print(np.array([[f'tau({xt}->{xtp1})={cpds[k][xtp1][xt]:.3f}' for xt in self.val_X] for xtp1 in self.val_X]))
        k = 'emission_mat'
        print(k)
        print(np.array([[f'e({xt}->{ot})={cpds[k][xt][ot]:.3f}' for ot in self.val_O] for xt in self.val_X]))

    ########################################
    ##########      Sampling      ##########
    ########################################
    def sample(self, N=1):
        """
        TODO sample N samples from the HMM.
        Assumes that the HMM CPDs are defined.
        :param N: optional, default=1. Number of samples.
        :return: (hidden, obs) for N samples from the HMM. shape of hidden & obs = (N,hmm.T)
        """
        cpds = self.get_CPDs()

        hidden_samples = np.zeros((N, self.T), dtype=int)
        obs_samples = np.zeros((N, self.T), dtype=int)
        
        for n in range(N): 
            hidden_samples[n, 0] = np.random.choice(self.val_X, p=cpds['prior'])
            obs_samples[n, 0] = np.random.choice(self.val_O, p=cpds['emission_mat'][hidden_samples[n, 0]])

            for t in range(1, self.T):
                hidden_samples[n, t] = np.random.choice(self.val_X, p=cpds['transition_mat'][hidden_samples[n, t-1]])
                obs_samples[n, t] = np.random.choice(self.val_O, p=cpds['emission_mat'][hidden_samples[n, t]])

        return (hidden_samples, obs_samples)

    
        

    ########################################
    ##########     Calc Prob.     ##########
    ########################################
    def log_joint(self, hidden, obs):
        """
        TODO calculate the log joint probability of the hidden, observations sequences for each sample p(x1:T[t],o1:T[i]).
        :param hidden - N hidden sequences. shape = (N,T)
        :param obs - N observations. shape = (N,T)
        :return log-joint probability. shape = (N)
        """
        pass

    def naive_log_likelihood(self, obs):
        """
        Calculate the log likelihood of the observations for each sample p(o1:T[i]) in a naive way (going over all
        possibilities). This will take many resources for T>5.
        :param obs - N observations. shape = (N,T)
        :return log-likelihood. shape = (N)
        """
        assert self.T < 6
        X = set(itertools.permutations(np.array([[x] * self.T for x in self.val_X]).flatten(), self.T))
        p = []
        for x in X:
            p.append(self.log_joint(np.broadcast_to(x, obs.shape), obs))
        return logsumexp(p, axis=0)

    def log_likelihood(self, obs):
        """
        TODO calculate the log likelihood of the observations for each sample p(o1:T[i]).
             Use the supplied forward-algorithm.
        :param obs - N observations. shape = (N,T)
        :return log-likelihood. shape = (N)
        """
        pass

    def log_prior_Xt(self):
        """
        TODO calculate the point-wise prior p(X_t=x)
        :return point-wise prior. shape = (T, |val(X)|)
        """
        log_p_Xt = None
        # tests
        assert_dist_non_negative(log_p_Xt)   # p(Xt = k) >= 0
        assert_dist_sums_to_1(log_p_Xt, axis=-1)  # sum_k[ p(Xt = k) ] == 1
        return log_p_Xt

    def log_naive_posterior_Xt(self, obs):
        """
        TODO calculate the point-wise posterior p(X_t=x | ot=obs[i][t])
        :param obs - N observations. shape = (N,T)
        :return point-wise posterior. shape = (N, T, |val(X)|)
        """
        log_posterior_Xt_given_Ot = None
        # tests
        assert_dist_non_negative(log_posterior_Xt_given_Ot)  # p(Xt = k | ot) >= 0
        assert_dist_sums_to_1(log_posterior_Xt_given_Ot, axis=-1)  # sum_k[ p(Xt = k | ot) ] == 1
        return log_posterior_Xt_given_Ot

    ########################################
    ##########     Prediction     ##########
    ########################################
    def naive_predict_by_naive_posterior(self, obs):
        """
        TODO predict a sequence of hidden states for each sample using the point-wise posterior:
            X_hat[i][t] = argmax_x[ p(X_t=x | ot=obs[i][t]) ]
        :param obs - N observations. shape = (N,T)
        :return X_hat - N hidden sequences. shape = (N,T)
        """
        pass


########################################
########################################
########################################
########################################
########################################
########################################
########################################
########################################



import os
# from HMM_proj1 import *
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


########################################
##########        Utils       ##########
########################################
def load_data(file_name, data_path='data/'):
    return pd.read_csv(data_path + file_name + '.csv', index_col=0).values


def get_hmm(num_hmm):
    # define HMMs
    val_X = np.arange(2)
    val_O = np.arange(2)
    if num_hmm == 1:
        hmm_params = {
            'prior': np.array([0.849, 0.151]),
            'transition_mat': np.array([[0.75, 0.25],  # p(X_{t+1}=x | X_t=0)
                                        [0.25, 0.75]]),   # p(X_{t+1}=x | X_t=1)
            'emission_mat': np.array([[0.83, 0.17], # p(O_t=o | X_t = 0)
                                      [0.17, 0.83]]) # p(O_t=o | X_t = 1)
        }
        T = 1000

    elif num_hmm == 2:
        hmm_params = {
            'prior': np.array([1, 0]),
            'transition_mat': np.array([[0.999, 0.001],
                                        [0.005, 0.995]]),
            'emission_mat': np.array([[0.99, 0.01],
                                      [0.01, 0.99]])
        }
        T = 1000
    elif num_hmm == 3:
        hmm_params = {
            'prior': np.array([1, 0]),
            'transition_mat': np.array([[0.999, 0.001],
                                        [0, 1]]),
            'emission_mat': np.array([[0.99, 0.01],
                                      [0.01, 0.99]])
        }
        T = 1000

    hmm = HMM(T=T, val_X=val_X, val_O=val_O, prior=hmm_params['prior'],
               transition_mat=hmm_params['transition_mat'], emission_mat=hmm_params['emission_mat'])
    print(f'Load HMM{num_hmm}. CPDs:')
    hmm.print_CPDs()
    print()
    print()
    return hmm


########################################
##########         Q2         ##########
########################################
def plot_sequence(sequences, seq_name=None, fig_name='', figsize=(20, 3)):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.plot(sequences)
    ax.set_title(f'Sequence of {seq_name} variables, {sequences.shape[1]} samples')
    ax.set_xlabel('T')
    ax.set_ylabel('modification status')
    fig.tight_layout()
    plt.savefig(f'plots/{fig_name}_sequence_{seq_name.lower()}', bbox_inches='tight')
    plt.show()


def plot_heatmap(table, table_name=None, fig_name='', figsize=(8, 4)):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    sns.heatmap(table, cmap='YlOrRd', ax=ax)
    ax.set_title(f'{table_name} variables, {table.shape[0]} samples')
    ax.set_xlabel('T')
    ax.set_ylabel('samples')
    fig.tight_layout()
    plt.savefig(f'plots/{fig_name}_heatmap_{table_name.lower()}', bbox_inches='tight')
    plt.show()


def plot_histogram(scores, score_names=None, fig_name='', bins=20):
    fig, ax = plt.subplots(1, 1, figsize=(4, 3))
    ax.hist(scores, bins=bins, density=True)
    ax.set_title(f'{score_names} histogram')
    ax.set_xlabel('log-joint probability')
    ax.set_ylabel('density')
    fig.tight_layout()
    plt.savefig(f'plots/{fig_name}_hist_{score_names.lower()}', bbox_inches='tight')
    plt.show()


def plot_samples(obs, hidden=None, N_plot=3, fig_name='hmm'):
    if hidden is not None: plot_sequence(hidden[:N_plot].T, seq_name=f'hidden', fig_name=fig_name)
    plot_sequence(obs[:N_plot].T, seq_name=f'observed', fig_name=fig_name)
    plot_heatmap(obs, table_name='Observed', fig_name=fig_name)


def Q2_sampling(hmm, fig_name='hmm', N=100):
    """
    Samples N samples from the HMM. Calculates the log joint probability of each sample.
    Plots:
        1. The hidden sequence of 3 samples.
        2. The observed sequence of 3 samples.
        3. A heatmap for all the observed samples.
        4. A histogram of the log joint probabilities.
    """
    hidden, obs = hmm.sample(N)
    log_joint = hmm.log_joint(hidden, obs)
    plot_samples(obs, hidden, fig_name=fig_name)
    plot_histogram(log_joint, score_names='Log-joint', bins=20, fig_name=fig_name)


########################################
##########         Q4         ##########
########################################
def Q4_identify_corrupt_data(hmm):
    """
    Loads the data and plots the histograms. Rest is TODO.
    Your job is to compute the validation_marginal_log_likelihood, real_marginal_log_likelihood and
    corrupt_marginal_log_likelihood below.
    """
    # get data
    validation_data = load_data('validation_data')
    test_data = load_data('test_data')
    '''
    TODO. Calculate marginal_log_likelihood on validation data, define prediction rule, 
            and calculate marginal_log_likelihood of test samples classified as real and as corrupted.
    '''
    validation_marginal_log_likelihood = None
    real_marginal_log_likelihood = None
    corrupt_marginal_log_likelihood = None

    # plot histograms
    plt.title('Histogram of marginal log-likelihood')
    mi = np.min([corrupt_marginal_log_likelihood.min(), real_marginal_log_likelihood.min(),
                 validation_marginal_log_likelihood.min()])
    _, bins, _ = plt.hist(validation_marginal_log_likelihood, label='validation data', bins=np.arange(mi - 10, 0, 4))
    plt.hist(real_marginal_log_likelihood, label='real test data', bins=bins)
    plt.hist(corrupt_marginal_log_likelihood, label='corrupted test data', bins=bins)
    plt.xlabel('marginal log-likelihood')
    plt.ylabel('frequency')
    plt.legend()
    plt.savefig('plots/Q4_hist', bbox_inches='tight')
    plt.show()


########################################
##########         Q5         ##########
########################################
def accuracy(true, pred):
    return np.mean(true == pred)


def test_pred(hmm, data_obs, data_hidden, val_X, invalid_transitions=None):
    def check_invalid_transitions(X_hat):
        if invalid_transitions is not None:
            for (xt, xtp1) in invalid_transitions:
                print(f'# invalid transitions from {xt} to {xtp1}')
                print(np.sum([((X_hat[:, t] == 1) & (X_hat[:, t + 1] == 0)).sum() for t in range(hmm.T - 1)]))

    X_hat = hmm.naive_predict_by_naive_posterior(data_obs)
    print(f'Naive prediction accuracy = {accuracy(data_hidden.flatten(), X_hat.flatten())}, confusion mat:')
    print(confusion_matrix(data_hidden.flatten(), X_hat.flatten(), labels=val_X))
    check_invalid_transitions(X_hat)


def Q5_naive_predication(hmm, N=100):
    hidden, obs = hmm.sample(N)
    test_pred(hmm, obs, hidden, hmm.val_X, invalid_transitions=[(1, 0)])

# 
# if __name__ == '__main__':
#     if not os.path.exists('plots/'): os.mkdir('plots/')
# 
#     # Q2
#     print('''
#         ########################################
#         ##########         Q2         ##########
#         ########################################
#     ''')
#     hmm1 = get_hmm(1)
#     Q2_sampling(hmm1, fig_name='Q2_hmm1')
#     hmm2 = get_hmm(2)
#     Q2_sampling(hmm2, fig_name='Q2_hmm2')
#     small_data = load_data('small_binary_data')
#     plot_samples(small_data, fig_name='Q2_real_data')
# 
#     # Q4
#     print('''
#         ########################################
#         ##########         Q4         ##########
#         ########################################
#     ''')
#     hmm2 = get_hmm(2)
#     Q4_identify_corrupt_data(hmm2)
# 
#     # Q5
#     print('''
#         ########################################
#         ##########         Q5         ##########
#         ########################################
#     ''')
#     hmm3 = get_hmm(3)
#     Q5_naive_predication(hmm3)


# Q2
print('''
    ########################################
    ##########         Q2         ##########
    ########################################
''')
hmm1 = get_hmm(1)
Q2_sampling(hmm1, fig_name='Q2_hmm1')


# hmm2 = get_hmm(2)
# Q2_sampling(hmm2, fig_name='Q2_hmm2')
# small_data = load_data('small_binary_data')
# plot_samples(small_data, fig_name='Q2_real_data')



    ########################################
    ##########         Q2         ##########
    ########################################

Load HMM1. CPDs:
prior
['prior(0)=0.849' 'prior(1)=0.151']
transition_mat
[['tau(0->0)=0.750' 'tau(1->0)=0.250']
 ['tau(0->1)=0.250' 'tau(1->1)=0.750']]
emission_mat
[['e(0->0)=0.830' 'e(0->1)=0.170']
 ['e(1->0)=0.170' 'e(1->1)=0.830']]




KeyboardInterrupt: 